In [7]:
import openff.toolkit
openff.toolkit.__version__

'0.16.6'

In [8]:
from openff.toolkit import Molecule, ForceField, unit
from openff.interchange.components._packmol import solvate_topology

In [9]:
import json
import numpy as np
from rdkit import Chem
from openeye import oeomega,oechem

from pathlib import Path

In [10]:
mol_ids = [
    #'mobley_7326706',
    # 'mobley_6474572',
    # 'mobley_6257907',
    # 'mobley_2178600',
    # 'mobley_4850657',
    # 'mobley_8117218',
    # 'mobley_7047032',
    # 'mobley_2523689',
    # 'mobley_4603202',
    # 'mobley_2481002'

    # 'mobley_9557440',
    # 'mobley_6195751',
    # 'mobley_6334915',

    'mobley_9015240'
    
]

In [22]:
with open("database.json", 'r') as infile:
    db = json.load(infile)

In [16]:
mols = []
for mol_id in mol_ids:
    offmol = Molecule.from_rdkit(Chem.AddHs(Chem.MolFromSmiles(f"{db[mol_id]['smiles']} {mol_id}")))

    oemol = Molecule.to_openeye(offmol)

    omega = oeomega.OEOmega()
    omega.SetMaxConfs(1)
    omega.SetIncludeInput(False)

    status = omega(oemol)

    offmol_conf = Molecule.from_openeye(oemol)

    #offmol_conf.assign_partial_charges('am1bccelf10')

    mols.append(offmol_conf)

print(mols)

[Molecule with name 'mobley_9015240' and SMILES '[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])Br']


In [19]:
ff = ForceField("vsites-v3_n2r-vdw-dimers_hmix-only-v2-torsions.offxml")
mol = mols[0]
ic = ff.create_interchange(topology=mol.to_topology())

In [21]:
ic.box = np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]])

ic.to_gro(str("/Users/megosato/Desktop/solvent.gro"))
ic.to_top(str("/Users/megosato/Desktop/solvent.top"))

In [17]:
# assign parameters using openff toolkit -> openff interchange
for mol in mols:

    print(mol)
    
    ff = ForceField("vsites-v3_n2r-vdw-dimers_hmix-only-v2-torsions.offxml")

    d = 1.6
    box_shape = np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]])
    solv_top = solvate_topology(mol.to_topology(), tolerance=1.0 * unit.angstrom, box_shape=box_shape)



    # RENAME THE MOLECULES
    for m in solv_top.molecules:
        if m.to_smiles() == '[H]O[H]':
            for atom in m.atoms:
                atom.metadata["residue_name"] = "HOH"     
            m.name = "HOH"
            
        elif m.to_smiles() == '[Na+]':
            for atom in m.atoms:
                atom.metadata["residue_name"] = "NA"
            m.name = "NA"
        elif m.to_smiles() == '[Cl-]':
            for atom in m.atoms:
                atom.metadata["residue_name"] = "CL"
            m.name = 'CL'
        else:
            for atom in m.atoms:
                atom.metadata["residue_name"] = "UNK"
            m.name = 'UNK'

    ic = ff.create_interchange(topology=solv_top)

    save_dir = Path(f"./vsites/{mol.name}")
    save_dir.mkdir(exist_ok=True, parents=True)

    ic.to_gro(str(save_dir / "solvent.gro"))
    ic.to_top(str(save_dir / "solvent.top"))
    

Molecule with name 'mobley_9015240' and SMILES '[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])Br'
